# Imports


In [8]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.formula.api as smf

import acareeg
from infant_connectivity import logit, add_bin_dist

# Global variables and configuration

In [9]:
palette = sns.color_palette(["green", "orange", "red"])

#con_path = Path("/Volumes/usc_data/ElementSE/eegip/con_paper/")
con_path = Path("/Users/christian/Documents/con_paper/")

plt.rcParams['figure.facecolor'] = 'white'

# Loading data

In [10]:
band = "broadband"
dat = pd.read_csv("analysis_data.csv")  # From notebok 2.1

### Confirmatory model (i.e., smaller simpler model, with less statistical power but less possible pitfalls)

In [11]:
"""tmp = dat[~dat.outliers_logit].groupby(["site", "age", "group", "subject", "sex"], observed=True).mean().reset_index()
tmp["log_con"] = logit(tmp.con)
tmp = tmp.groupby(["site", "group", "subject", "sex"],
                  observed=True).mean().reset_index()


formula = "log_con ~ group*sex + site*age"
md = smf.ols(formula, tmp)
#table = sm.stats.anova_lm(md.fit(), typ=3)
#table
md.fit().pvalues, md.fit().params
""";

In [12]:
tmp = dat[~dat.outliers_logit].groupby(["site", "age", "group", "subject", "sex"], observed=True).mean().reset_index()
tmp["log_con"] = logit(tmp.con)

# Model (2)
formula = "log_con ~ (group + sex + site + age)**2"
md = smf.mixedlm(formula, tmp, groups="subject_no")
md = md.fit()

print(f"beta={md.params['age']:.4f}; p={md.pvalues['age']:.6f}")
md.summary()

beta=-0.0148; p=0.000018


/Users/christian/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
                    Mixed Linear Model Regression Results
==============================================================================
Model:                    MixedLM         Dependent Variable:         log_con 
No. Observations:         288             Method:                     REML    
No. Groups:               176             Scale:                      0.0083  
Min. group size:          1               Log-Likelihood:             234.1339
Max. group size:          2               Converged:                  Yes     
Mean group size:          1.6                                                 
------------------------------------------------------------------------------
                                   Coef.  Std.Err.    z    P>|z| [0.025 0.975]
------------------------------------------------------------------------------
Intercept                          -1.471    0.034 -43.379 0.000 -1.537 -1.404
group[T.HRA-ASD]                   -0.061    0.061  -1.002 0.316 -0.180  0.058
group[T.HRA-noASD]                 -0.011    0.042  -0.269 0.788 -0.093  0.070
sex[T.M]                           -0.011    0.040  -0.282 0.778 -0.090  0.067
site[T.Seattle]                     0.061    0.038   1.594 0.111 -0.014  0.137
group[T.HRA-ASD]:sex[T.M]           0.034    0.034   0.985 0.325 -0.033  0.101
group[T.HRA-noASD]:sex[T.M]        -0.012    0.026  -0.480 0.631 -0.063  0.038
group[T.HRA-ASD]:site[T.Seattle]    0.009    0.034   0.249 0.803 -0.059  0.076
group[T.HRA-noASD]:site[T.Seattle]  0.029    0.026   1.127 0.260 -0.022  0.080
sex[T.M]:site[T.Seattle]           -0.003    0.023  -0.129 0.897 -0.049  0.043
age                                -0.015    0.003  -4.291 0.000 -0.022 -0.008
group[T.HRA-ASD]:age                0.004    0.005   0.698 0.485 -0.007  0.014
group[T.HRA-noASD]:age              0.000    0.004   0.124 0.901 -0.007  0.008
sex[T.M]:age                       -0.000    0.004  -0.022 0.983 -0.007  0.007
site[T.Seattle]:age                -0.002    0.004  -0.502 0.616 -0.009  0.005
subject_no Var                      0.000    0.009                            
==============================================================================

"""

In [13]:
# Model (3)

tmp2 = tmp[(~np.isnan(tmp.adoscss_latest)) & (tmp.group != "Control")]
formula = "log_con ~ (adoscss_earliest + sex + site + age)**2"
md = smf.mixedlm(formula, tmp2, groups="subject_no")
md = md.fit()

print(f"beta={md.params['age']:.4f}; p={md.pvalues['age']:.6f}")
md.summary()

beta=-0.0112; p=0.055155


/Users/christian/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
                   Mixed Linear Model Regression Results
============================================================================
Model:                     MixedLM        Dependent Variable:        log_con
No. Observations:          112            Method:                    REML   
No. Groups:                70             Scale:                     0.0061 
Min. group size:           1              Log-Likelihood:            79.5684
Max. group size:           2              Converged:                 Yes    
Mean group size:           1.6                                              
----------------------------------------------------------------------------
                                 Coef.  Std.Err.    z    P>|z| [0.025 0.975]
----------------------------------------------------------------------------
Intercept                        -1.503    0.062 -24.088 0.000 -1.625 -1.381
sex[T.M]                         -0.119    0.062  -1.918 0.055 -0.241  0.003
site[T.Seattle]                   0.153    0.068   2.244 0.025  0.019  0.286
sex[T.M]:site[T.Seattle]         -0.018    0.040  -0.446 0.656 -0.097  0.061
adoscss_earliest                  0.010    0.013   0.820 0.412 -0.014  0.035
adoscss_earliest:sex[T.M]         0.008    0.008   1.042 0.297 -0.007  0.023
adoscss_earliest:site[T.Seattle] -0.006    0.009  -0.688 0.492 -0.025  0.012
age                              -0.011    0.006  -1.918 0.055 -0.023  0.000
sex[T.M]:age                      0.010    0.005   1.873 0.061 -0.000  0.020
site[T.Seattle]:age              -0.007    0.006  -1.101 0.271 -0.018  0.005
adoscss_earliest:age             -0.002    0.001  -1.387 0.165 -0.004  0.001
subject_no Var                    0.001    0.017                            
============================================================================

"""

#### Equivalent models but without averaging (supplementation documents)

In [14]:
tmp = dat[~dat.outliers_logit]#.groupby(["site", "age", "group", "subject", "sex"], observed=True).mean().reset_index()
#tmp = dat[~dat.outliers_logit].groupby(["site", "age", "group", "subject", "sex"], observed=True).mean().reset_index()
tmp["log_con"] = logit(tmp.con)

# 3.a
formula = "log_con ~ (group + sex + site + age)**2"
md = smf.mixedlm(formula, tmp, groups="subject_no")
print(f"Number of recordings: {tmp.shape[0]}; Number of subjects: {tmp.groupby('subject_no').mean().shape[0]}")
md = md.fit()
md.summary()


Number of recordings: 599040; Number of subjects: 176


/Users/christian/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/christian/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/Users/christian/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/christian/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with cg
  warnings.warn(
/Users/christian/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization fai

<class 'statsmodels.iolib.summary2.Summary'>
"""
                     Mixed Linear Model Regression Results
===============================================================================
Model:                    MixedLM        Dependent Variable:        log_con    
No. Observations:         599040         Method:                    REML       
No. Groups:               176            Scale:                     0.0756     
Min. group size:          2080           Log-Likelihood:            -77183.1460
Max. group size:          4160           Converged:                 No         
Mean group size:          3403.6                                               
-------------------------------------------------------------------------------
                                   Coef.  Std.Err.    z     P>|z| [0.025 0.975]
-------------------------------------------------------------------------------
Intercept                          -1.521    0.014 -110.906 0.000 -1.548 -1.494
group[T.HRA-ASD]                   -0.052    0.031   -1.677 0.094 -0.112  0.009
group[T.HRA-noASD]                 -0.011    0.019   -0.575 0.565 -0.049  0.027
sex[T.M]                           -0.015    0.019   -0.753 0.451 -0.052  0.023
site[T.Seattle]                     0.058    0.020    2.821 0.005  0.018  0.098
group[T.HRA-ASD]:sex[T.M]           0.046    0.034    1.345 0.179 -0.021  0.112
group[T.HRA-noASD]:sex[T.M]        -0.002    0.027   -0.060 0.953 -0.054  0.051
group[T.HRA-ASD]:site[T.Seattle]    0.008    0.034    0.235 0.814 -0.059  0.075
group[T.HRA-noASD]:site[T.Seattle]  0.020    0.027    0.759 0.448 -0.032  0.073
sex[T.M]:site[T.Seattle]            0.005    0.024    0.194 0.846 -0.042  0.051
age                                -0.011    0.000  -42.982 0.000 -0.012 -0.011
group[T.HRA-ASD]:age                0.002    0.000    5.198 0.000  0.001  0.003
group[T.HRA-noASD]:age             -0.001    0.000   -2.542 0.011 -0.001 -0.000
sex[T.M]:age                       -0.000    0.000   -0.276 0.782 -0.001  0.000
site[T.Seattle]:age                -0.001    0.000   -4.380 0.000 -0.002 -0.001
subject_no Var                      0.005    0.003                             
===============================================================================

"""

In [15]:
# 3.b

tmp2 = tmp[(~np.isnan(tmp.adoscss_latest)) & (tmp.group != "Control")]
formula = "log_con ~ (adoscss_earliest + sex + site + age)**2"
md = smf.mixedlm(formula, tmp2, groups="subject_no")

print(f"Number of recordings: {tmp2.shape[0]}; Number of subjects: {tmp2.groupby('subject_no').mean().shape[0]}")

md = md.fit()
md.summary()

Number of recordings: 232960; Number of subjects: 70


/Users/christian/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/christian/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/Users/christian/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
                   Mixed Linear Model Regression Results
============================================================================
Model:                   MixedLM       Dependent Variable:       log_con    
No. Observations:        232960        Method:                   REML       
No. Groups:              70            Scale:                    0.0712     
Min. group size:         2080          Log-Likelihood:           -23036.4207
Max. group size:         4160          Converged:                Yes        
Mean group size:         3328.0                                             
----------------------------------------------------------------------------
                                 Coef.  Std.Err.    z    P>|z| [0.025 0.975]
----------------------------------------------------------------------------
Intercept                        -1.562    0.024 -65.483 0.000 -1.609 -1.516
sex[T.M]                         -0.093    0.034  -2.761 0.006 -0.159 -0.027
site[T.Seattle]                   0.170    0.041   4.112 0.000  0.089  0.251
sex[T.M]:site[T.Seattle]         -0.013    0.040  -0.317 0.751 -0.090  0.065
adoscss_earliest                  0.008    0.006   1.359 0.174 -0.004  0.020
adoscss_earliest:sex[T.M]         0.007    0.007   1.005 0.315 -0.007  0.022
adoscss_earliest:site[T.Seattle] -0.005    0.009  -0.539 0.590 -0.023  0.013
age                              -0.009    0.001 -16.729 0.000 -0.010 -0.008
sex[T.M]:age                      0.008    0.000  18.680 0.000  0.007  0.009
site[T.Seattle]:age              -0.009    0.001 -17.720 0.000 -0.010 -0.008
adoscss_earliest:age             -0.001    0.000 -10.875 0.000 -0.001 -0.001
subject_no Var                    0.005    0.004                            
============================================================================

"""